In [1]:
import os
import json

root_json = '/workspaces/solidity/json'
for json_file in os.listdir(root_json):
    with open(os.path.join(root_json, json_file), 'r') as f:
        items = json.load(f)
    print(json_file, len(items))

smartbugs.json 143
error.json 388
clean.json 2354
solidifi.json 350


In [21]:
import os 

dir = '/workspaces/solidity/integrate_dataset'
for vuln in os.listdir(dir):
    if vuln == 'clean':
        continue
    path = os.path.join(dir, vuln, 'integrate', 'vulnerabilities.json')
    new_path = os.path.join(dir, vuln, 'integrate', 'vuln_vulnerabilities.json')
    os.rename(path, new_path)

In [2]:
import json

path_error = '/workspaces/solidity/error.json'
path_clean = '/workspaces/solidity/integrate_dataset/clean/clean_vulnerabilities.json'

with open(path_error, 'r') as f:
    item_error = json.load(f)

with open(path_clean, 'r') as f:
    item_clean = json.load(f)

for error in item_error:
    for clean in item_clean:
        if error['name'] == clean['name']:
            clean['version'] = '0.5.0'

with open(path_clean, 'w') as f:
     json.dump(item_clean, f)


In [22]:
import os
import networkx as nx

cfg_path = '/workspaces/solidity/integrate_dataset/other/integrate/cfg.gpickle'
cfg = nx.read_gpickle(cfg_path)